# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f29693f2460>
├── 'a' --> tensor([[-0.0997, -1.1201,  1.3680],
│                   [-0.1333,  0.9772,  0.9405]])
└── 'x' --> <FastTreeValue 0x7f29693f23a0>
    └── 'c' --> tensor([[-1.0892, -0.1569, -0.0499,  0.1720],
                        [ 1.2973, -1.1874,  0.6996,  0.0174],
                        [ 0.5678, -1.1513,  1.1284, -0.2079]])

In [4]:
t.a

tensor([[-0.0997, -1.1201,  1.3680],
        [-0.1333,  0.9772,  0.9405]])

In [5]:
%timeit t.a

68.3 ns ± 1.02 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f29693f2460>
├── 'a' --> tensor([[ 0.2624,  1.4128, -0.0393],
│                   [ 0.5302, -0.0597, -0.8609]])
└── 'x' --> <FastTreeValue 0x7f29693f23a0>
    └── 'c' --> tensor([[-1.0892, -0.1569, -0.0499,  0.1720],
                        [ 1.2973, -1.1874,  0.6996,  0.0174],
                        [ 0.5678, -1.1513,  1.1284, -0.2079]])

In [7]:
%timeit t.a = new_value

70.6 ns ± 2.78 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.0997, -1.1201,  1.3680],
               [-0.1333,  0.9772,  0.9405]]),
    x: Batch(
           c: tensor([[-1.0892, -0.1569, -0.0499,  0.1720],
                      [ 1.2973, -1.1874,  0.6996,  0.0174],
                      [ 0.5678, -1.1513,  1.1284, -0.2079]]),
       ),
)

In [10]:
b.a

tensor([[-0.0997, -1.1201,  1.3680],
        [-0.1333,  0.9772,  0.9405]])

In [11]:
%timeit b.a

60.7 ns ± 2.43 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4612, -1.1346, -0.3292],
               [-1.8448, -1.0110,  1.0477]]),
    x: Batch(
           c: tensor([[-1.0892, -0.1569, -0.0499,  0.1720],
                      [ 1.2973, -1.1874,  0.6996,  0.0174],
                      [ 0.5678, -1.1513,  1.1284, -0.2079]]),
       ),
)

In [13]:
%timeit b.a = new_value

588 ns ± 18.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

924 ns ± 21.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.4 µs ± 169 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

266 µs ± 15.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

266 µs ± 20.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2964317520>
├── 'a' --> tensor([[[-0.0997, -1.1201,  1.3680],
│                    [-0.1333,  0.9772,  0.9405]],
│           
│                   [[-0.0997, -1.1201,  1.3680],
│                    [-0.1333,  0.9772,  0.9405]],
│           
│                   [[-0.0997, -1.1201,  1.3680],
│                    [-0.1333,  0.9772,  0.9405]],
│           
│                   [[-0.0997, -1.1201,  1.3680],
│                    [-0.1333,  0.9772,  0.9405]],
│           
│                   [[-0.0997, -1.1201,  1.3680],
│                    [-0.1333,  0.9772,  0.9405]],
│           
│                   [[-0.0997, -1.1201,  1.3680],
│                    [-0.1333,  0.9772,  0.9405]],
│           
│                   [[-0.0997, -1.1201,  1.3680],
│                    [-0.1333,  0.9772,  0.9405]],
│           
│                   [[-0.0997, -1.1201,  1.3680],
│                    [-0.1333,  0.9772,  0.9405]]])
└── 'x' --> <FastTreeValue 0x7f28afbe3bb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

42.7 µs ± 1.51 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f29643607c0>
├── 'a' --> tensor([[-0.0997, -1.1201,  1.3680],
│                   [-0.1333,  0.9772,  0.9405],
│                   [-0.0997, -1.1201,  1.3680],
│                   [-0.1333,  0.9772,  0.9405],
│                   [-0.0997, -1.1201,  1.3680],
│                   [-0.1333,  0.9772,  0.9405],
│                   [-0.0997, -1.1201,  1.3680],
│                   [-0.1333,  0.9772,  0.9405],
│                   [-0.0997, -1.1201,  1.3680],
│                   [-0.1333,  0.9772,  0.9405],
│                   [-0.0997, -1.1201,  1.3680],
│                   [-0.1333,  0.9772,  0.9405],
│                   [-0.0997, -1.1201,  1.3680],
│                   [-0.1333,  0.9772,  0.9405],
│                   [-0.0997, -1.1201,  1.3680],
│                   [-0.1333,  0.9772,  0.9405]])
└── 'x' --> <FastTreeValue 0x7f28b625df70>
    └── 'c' --> tensor([[-1.0892, -0.1569, -0.0499,  0.1720],
                        [ 1.2973, -1.1874,  0.6996,  0.0174],
                 

In [23]:
%timeit t_cat(trees)

40.3 µs ± 1.64 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

74.1 µs ± 2.73 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.0892, -0.1569, -0.0499,  0.1720],
                       [ 1.2973, -1.1874,  0.6996,  0.0174],
                       [ 0.5678, -1.1513,  1.1284, -0.2079]],
              
                      [[-1.0892, -0.1569, -0.0499,  0.1720],
                       [ 1.2973, -1.1874,  0.6996,  0.0174],
                       [ 0.5678, -1.1513,  1.1284, -0.2079]],
              
                      [[-1.0892, -0.1569, -0.0499,  0.1720],
                       [ 1.2973, -1.1874,  0.6996,  0.0174],
                       [ 0.5678, -1.1513,  1.1284, -0.2079]],
              
                      [[-1.0892, -0.1569, -0.0499,  0.1720],
                       [ 1.2973, -1.1874,  0.6996,  0.0174],
                       [ 0.5678, -1.1513,  1.1284, -0.2079]],
              
                      [[-1.0892, -0.1569, -0.0499,  0.1720],
                       [ 1.2973, -1.1874,  0.6996,  0.0174],
                       [ 0.5678, -1.1513,  1.1284, -0.2079]],

In [26]:
%timeit Batch.stack(batches)

99.2 µs ± 1.63 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.0892, -0.1569, -0.0499,  0.1720],
                      [ 1.2973, -1.1874,  0.6996,  0.0174],
                      [ 0.5678, -1.1513,  1.1284, -0.2079],
                      [-1.0892, -0.1569, -0.0499,  0.1720],
                      [ 1.2973, -1.1874,  0.6996,  0.0174],
                      [ 0.5678, -1.1513,  1.1284, -0.2079],
                      [-1.0892, -0.1569, -0.0499,  0.1720],
                      [ 1.2973, -1.1874,  0.6996,  0.0174],
                      [ 0.5678, -1.1513,  1.1284, -0.2079],
                      [-1.0892, -0.1569, -0.0499,  0.1720],
                      [ 1.2973, -1.1874,  0.6996,  0.0174],
                      [ 0.5678, -1.1513,  1.1284, -0.2079],
                      [-1.0892, -0.1569, -0.0499,  0.1720],
                      [ 1.2973, -1.1874,  0.6996,  0.0174],
                      [ 0.5678, -1.1513,  1.1284, -0.2079],
                      [-1.0892, -0.1569, -0.0499,  0.1720],
                   

In [28]:
%timeit Batch.cat(batches)

175 µs ± 3.17 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

539 µs ± 11.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
